In [1]:
from IPython.display import clear_output
!pip install ensemble-boxes
clear_output()

In [2]:
import pandas as pd
import numpy as np
import csv
import os
import random
from ensemble_boxes import weighted_boxes_fusion
import torch
from pathlib import Path

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

In [3]:
def apply_wbf_and_save_final_submission(predictions, image_ids, iou_thr, skip_box_thr, output_path="submission_wbf.csv"):
    wbf_results = []

    for image_id in image_ids:
        all_boxes, all_scores, all_labels = [], [], []

        for model_preds in predictions.values():
            for size_preds in model_preds.values():
                if image_id not in size_preds:
                    continue
                pred = size_preds[image_id]
                if not pred["boxes"]:
                    continue
                all_boxes.append(pred["boxes"])
                all_scores.append(pred["scores"])
                all_labels.append(pred["labels"])

        if not all_boxes:
            pred_str = "no boxes"
        else:
            fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
                all_boxes, all_scores, all_labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr
            )

            pred_str = " ".join(
                f"{int(lbl)} {score:.6f} {(b[0]+b[2])/2:.6f} {(b[1]+b[3])/2:.6f} {(b[2]-b[0]):.6f} {(b[3]-b[1]):.6f}"
                for b, score, lbl in zip(fused_boxes, fused_scores, fused_labels)
            )

        wbf_results.append({
            "image_id": image_id,
            "prediction_string": pred_str
        })

    wbf_df = pd.DataFrame(wbf_results)
    wbf_df.to_csv(output_path, index=False, quoting=csv.QUOTE_MINIMAL)
    print(f"[notice] ✅ WBF submission saved to {output_path}")
    # print(wbf_df.shape)
    print(wbf_df.head(10))


def load_prediction_structure(csv_root, selected_keys, selected_indices, selected_sizes):
    """
    Loads predictions[model_key][image_size][image_id] structure from saved CSVs.
    """
    prediction_data = {}
    csv_root = Path(csv_root)

    for model_key in selected_keys:
        model_dir = csv_root / model_key
        prediction_data[model_key] = {}
        # print(model_dir)
        for model_index in selected_indices:
            model_dir_idx = model_dir / model_index
            prediction_data[model_key][model_index] = {}
            # print(model_dir)
            
            for image_size in selected_sizes:
                csv_file = model_dir_idx / f"{image_size}.csv"
                # print(csv_file)
                prediction_data[model_key][model_index][image_size]={}
                df = pd.read_csv(csv_file)
                image_pred_map = {}
    
                for _, row in df.iterrows():
                    image_id = row["image_id"]
                    pred_str = row["prediction_string"]
                    if pred_str.strip().lower() == "no boxes":
                        image_pred_map[image_id] = {"boxes": [], "scores": [], "labels": []}
                        continue
    
                    values = list(map(float, pred_str.strip().split()))
                    boxes, scores, labels = [], [], []
                    for i in range(0, len(values), 6):
                        cls, conf, x, y, w, h = values[i:i+6]
                        if not (0 <= x <= 1 and 0 <= y <= 1 and 0 <= w <= 1 and 0 <= h <= 1):
                            print(f"[before] Box out of bounds in {csv_file.name} for image {image_id}: {[x, y, w, h]}")

                        # we have to use min max due to floating-point precision issues
                        x1 = max(0.0, x - w / 2)
                        y1 = max(0.0, y - h / 2)
                        x2 = min(1.0, x + w / 2)
                        y2 = min(1.0, y + h / 2)

                        if not (0 <= x1 <= 1 and 0 <= x2 <= 1 and 0 <= y1 <= 1 and 0 <= y2 <= 1):
                            print(f"[after] Box out of bounds in {csv_file.name} for image {image_id}: {[x1, y1, x2, y2]} : {[x, y, w, h]}")

                        boxes.append([x1, y1, x2, y2])
                        scores.append(conf)
                        labels.append(int(cls))
    
                    image_pred_map[image_id] = {"boxes": boxes, "scores": scores, "labels": labels}
    
                prediction_data[model_key][model_index][image_size] = image_pred_map
    
    return prediction_data



In [4]:
# === Setup
csv_root_dir = "/kaggle/input/falcon-multiclass-yolo-inference" 
keys = ["default"]

indices= ["0", "1", "2", "3", "4"]
sizes= [640, 800, 1056, 1376, 1600, 1920]

iou_thresholds = [0.55]
skip_box_thresholds = [0]

# === Load selected predictions from CSVs
loaded_preds = load_prediction_structure(csv_root_dir, keys, indices, sizes)
print('Loaded all the selected predictions from CSVs')

# === Get image_ids from first found image set
def get_all_image_ids(loaded_preds):
    for key_data in loaded_preds.values():
        for idx_data in key_data.values():
            for size_data in idx_data.values():
                return list(size_data.keys())
    return []

image_ids = get_all_image_ids(loaded_preds)

# === Apply WBF for all combinations ===
for iou_thr in iou_thresholds:
    for skip_box_thr in skip_box_thresholds:
        for pred_key, pred_data in loaded_preds.items():
            suffix = f"{iou_thr}_{skip_box_thr}" if pred_key == "default" else f"{iou_thr}_{skip_box_thr}_{pred_key}"
            output_path = f"submission_wbf_{suffix}.csv"
            apply_wbf_and_save_final_submission(
                pred_data,
                image_ids,
                output_path=output_path,
                iou_thr=iou_thr,
                skip_box_thr=skip_box_thr
            )

Loaded all the selected predictions from CSVs
[notice] ✅ WBF submission saved to submission_wbf_0.55_0.csv
   image_id                                  prediction_string
0  IMG_8712  0 0.973890 0.460518 0.488594 0.261523 0.248254...
1  IMG_8949  0 0.990398 0.521679 0.514810 0.228140 0.253079...
2  IMG_8778  0 0.936878 0.426728 0.736757 0.521896 0.451697...
3  IMG_8723  0 0.954615 0.282394 0.508608 0.192953 0.304130...
4  IMG_8771  1 0.977931 0.704394 0.497752 0.181431 0.208853...
5  IMG_9164  1 0.985447 0.576494 0.714722 0.221684 0.152418...
6  IMG_9946  0 0.910370 0.278589 0.144289 0.223807 0.287968...
7  IMG_9953  1 0.984054 0.652963 0.620527 0.118348 0.178709...
8  IMG_8796  1 0.982464 0.348215 0.530921 0.193009 0.236973...
9  IMG_9951  1 0.707031 0.638451 0.604309 0.072457 0.195635...
